In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from itertools import product

import lightgbm as lgb
from tqdm import tqdm_notebook

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [9]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df


In [10]:
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [11]:
sales = sales_train

# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
all_data2 = all_data.copy()
del grid, gb 
gc.collect();

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


In [12]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [13]:
dates = all_data['date_block_num']

last_block = dates.max()

X_train = all_data.loc[dates <=  last_block].drop(to_drop_cols, axis=1)
X_train_part = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <=  last_block, 'target'].values
y_train_part = all_data.loc[dates <  last_block, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

# print(X_train_part.shape)
# print(y_train_part.shape)

X_train.to_pickle("X_train.pkl")
X_train_part.to_pickle("X_train_part.pkl")
X_test.to_pickle("X_test.pkl")

np.save("y_train.npy", y_train)
np.save("y_train_part.npy", y_train_part)
np.save("y_test.npy", y_test)

In [14]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data2.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

test_data = test.copy()
test_data['date_block_num'] = 34

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data2[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    test_data = pd.merge(test_data, train_shift, on=index_cols, how='left').fillna(0)

test_data = pd.merge(test_data, item_category_mapping, how='left', on='item_id')
test_data = downcast_dtypes(test_data)
gc.collect();


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [15]:
def lgbm_train(X_train,y_train,X_test,y_test,num_iter):
    lgb_params = {
                   'feature_fraction': 0.75,
                   'metric': 'rmse',
                   'nthread':1, 
                   'min_data_in_leaf': 2**7, 
                   'bagging_fraction': 0.75, 
                   'learning_rate': 0.03, 
                   'objective': 'mse', 
                   'bagging_seed': 2**7, 
                   'num_leaves': 2**7,
                   'bagging_freq':1,
                   'verbose':0 
                  }

    evals_result = {}
    
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_valid = lgb.Dataset(X_test, label=y_test)
    model = lgb.train(lgb_params, lgb_train, num_iter,
                      valid_sets=[lgb_train, lgb_valid],
                      evals_result=evals_result,
                      verbose_eval=10)
    return model,evals_result


In [18]:

X_train = pd.read_pickle("/kaggle/input/predict-future-sales/X_train.pkl")
X_train_part = pd.read_pickle("/kaggle/input/predict-future-sales/X_train_part.pkl")
X_test =  pd.read_pickle("/kaggle/input/predict-future-sales/X_test.pkl")

y_train = np.load("/kaggle/input/predict-future-sales/y_train.npy")
y_train_part = np.load("/kaggle/input/predict-future-sales/y_train_part.npy")
y_test =  np.load("/kaggle/input/predict-future-sales/y_test.npy")


In [ ]:
y_train_part = y_train_part.clip(0,40)

print(X_train_part.shape)
print(y_train_part.shape)
model,evals_result  = lgbm_train(X_train_part, y_train_part, X_test, y_test, 200)

(6186922, 21)
(6186922,)
[10]	training's rmse: 1.28027	valid_1's rmse: 5.27475
[20]	training's rmse: 1.17893	valid_1's rmse: 5.22955
[30]	training's rmse: 1.11709	valid_1's rmse: 5.2001
[40]	training's rmse: 1.07662	valid_1's rmse: 5.18031
[50]	training's rmse: 1.05003	valid_1's rmse: 5.16596


In [22]:
# full model traininig for submission
y_train = y_train.clip(0,40)

print(X_train.shape)
print(y_train.shape)
model_full, evals_result  = lgbm_train(X_train, y_train, X_test, y_test, 200) 

(6425094, 21)
(6425094,)
[10]	training's rmse: 1.28088	valid_1's rmse: 5.27328
[20]	training's rmse: 1.18107	valid_1's rmse: 5.2259
[30]	training's rmse: 1.12012	valid_1's rmse: 5.19518
[40]	training's rmse: 1.08018	valid_1's rmse: 5.17332
[50]	training's rmse: 1.05399	valid_1's rmse: 5.15705
[60]	training's rmse: 1.03723	valid_1's rmse: 5.14567
[70]	training's rmse: 1.02489	valid_1's rmse: 5.13689
[80]	training's rmse: 1.0156	valid_1's rmse: 5.13003
[90]	training's rmse: 1.00835	valid_1's rmse: 5.12419
[100]	training's rmse: 1.00199	valid_1's rmse: 5.12054
[110]	training's rmse: 0.997315	valid_1's rmse: 5.11699
[120]	training's rmse: 0.992634	valid_1's rmse: 5.11384
[130]	training's rmse: 0.988291	valid_1's rmse: 5.11204
[140]	training's rmse: 0.984292	valid_1's rmse: 5.10975
[150]	training's rmse: 0.980693	valid_1's rmse: 5.1078
[160]	training's rmse: 0.97772	valid_1's rmse: 5.10654
[170]	training's rmse: 0.974991	valid_1's rmse: 5.10579
[180]	training's rmse: 0.972717	valid_1's rmse

In [ ]:
model.save_model('lgbm_model.txt')
model_full.save_model('lgbm_model_full.txt')

In [4]:
model = lgb.Booster(model_file='/kaggle/input/predict-future-sales/lgbm_model.txt')
model_full = lgb.Booster(model_file='/kaggle/input/predict-future-sales/lgbm_model_full.txt')

In [20]:
#linear regression in preparation for ensembling
lr = LinearRegression()
y_train_part = y_train_part.clip(0,40)
lr.fit(X_train_part.values, y_train_part)
print(X_test.shape)
print(y_test.shape)
pred_lr = lr.predict(X_test)
print('Test RMSE for linreg is %f' % np.sqrt(mean_squared_error(y_test, pred_lr)))


(238172, 21)
(238172,)
Test RMSE for linreg is 5.181381


In [6]:
pred_lgb = model.predict(X_test)
X_test_level2 = np.c_[pred_lr, pred_lgb]
np.save("X_test_level2.npy", X_test_level2)

In [ ]:

dates_train = dates[dates <  last_block]
dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train_part[dates_train.isin([27, 28, 29, 30, 31, 32])]

# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 2])

count = 0
# Now fill `X_train_level2` with metafeatures
for cur_block_num in [27, 28, 29, 30, 31, 32]:
    
    print(cur_block_num)
    
    '''
        1. Split `X_train` into parts
           Remember, that corresponding dates are stored in `dates_train` 
        2. Fit linear regression 
        3. Fit LightGBM and put predictions          
        4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
           You can use `dates_train_level2` for it
           Make sure the order of the meta-features is the same as in `X_test_level2`
    '''
    X_train_cur = X_train_part[dates_train.isin(range(12,cur_block_num))]
    y_train_cur = y_train_part[dates_train.isin(range(12,cur_block_num))]
    X_test_cur = X_train_part[dates_train.isin([cur_block_num])]
    
    lr.fit(X_train_cur.values, y_train_cur)
    pred_lr = lr.predict(X_test_cur.values)
    
    model_temp,_ = lgbm_train(X_train_cur, y_train_cur, X_train_cur, y_train_cur, 200)
    pred_lgb = model_temp.predict(X_test_cur)
    
    for i in range(len(pred_lr)):
        X_train_level2[count+i][0]=pred_lr[i]
        X_train_level2[count+i][1]=pred_lgb[i]
    count+=len(pred_lr)
np.save("X_train_full_level2.npy", X_train_level2)
np.save("y_train_full_level2.npy", y_train_level2)

In [25]:
#Full linear regression
lr_full = LinearRegression()
y_train = y_train.clip(0,40)
lr_full.fit(X_train.values, y_train)
pred_lr_full = lr_full.predict(X_test)
print('Test RMSE for linreg is %f' % np.sqrt(mean_squared_error(y_test, pred_lr_full)))

Test RMSE for linreg is 5.229523


In [ ]:
alphas_to_try = np.linspace(0, 1, 1001)
best_rmse = 1000
best_alpha = -1
for alpha in alphas_to_try:
    curr_rmse = np.sqrt(mean_squared_error(y_train_level2, np.dot(X_train_level2, [alpha,1-alpha])))
    if curr_rmse < best_rmse:
        best_rmse = curr_rmse
        best_alpha = alpha
# YOUR CODE GOES HERE
best_alpha = best_alpha
print(best_rmse)
print(best_alpha)

In [16]:
X_train_level2 = np.load("/kaggle/input/predict-future-sales/X_train_level2.npy")
y_train_level2 = np.load("/kaggle/input/predict-future-sales/y_train_level2.npy")

lr.fit(X_train_level2, y_train_level2)

test_preds = lr.predict(X_test_level2)
rmse_test_stacking = np.sqrt(mean_squared_error(y_test, test_preds))
rmse_test_stacking

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/predict-future-sales/X_train_level2.npy'

In [ ]:
pred_lgb_test = model.predict(X_test)
pred_ens_test = best_alpha*pred_lgb_test + (1-best_alpha)*pred_lr


In [27]:
to_drop_cols = ['ID','date_block_num']
X_test_final =  test_data.drop(to_drop_cols, axis=1)
pred_lgb_final = model_full.predict(X_test_final)
# pred_lr_final = lr_full.predict(X_test_final)
#pred_ens = best_alpha*pred_lgb_final + (1-best_alpha)*pred_lr_final
#print(pred_lgb)

merged = test.copy()
merged['item_cnt_month'] = pred_lgb_final
merged['item_cnt_month']=merged['item_cnt_month'].clip(lower=0,upper=20)
merged=merged.drop(['shop_id','item_id'],axis=1)
merged['ID']=merged['ID'].astype('int')
merged.to_csv("lightgbm_clipto40.csv",index=False)
print(merged.head)

<bound method NDFrame.head of             ID  item_cnt_month
0            0        0.430201
1            1        0.195441
2            2        0.827994
3            3        0.288239
4            4        1.906440
...        ...             ...
214195  214195        0.124841
214196  214196        0.045553
214197  214197        0.021552
214198  214198        0.019733
214199  214199        0.019733

[214200 rows x 2 columns]>
